In [1]:
import os

In [2]:
%pwd

'd:\\my projects\\ML assigment\\ML-flow-deep-learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\my projects\\ML assigment\\ML-flow-deep-learning'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Vaibhav4228/ML-flow-deep-learning.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Vaibhav4228"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9af07288d31491162822c0a38a813088471c7b39"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:

from deepLearning.constants import *
from deepLearning.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data",
            mlflow_uri="https://dagshub.com/Vaibhav4228/ML-flow-deep-learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

d:\anaconda\envs\mlops311\Lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
import tensorflow as tf
import mlflow
import tempfile
import shutil
from urllib.parse import urlparse
from pathlib import Path
from deepLearning.utils.common import save_json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        dg_kwargs = dict(rescale=1.0/255, validation_split=0.30)
        df_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        gen = tf.keras.preprocessing.image.ImageDataGenerator(**dg_kwargs)
        self.valid_generator = gen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **df_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(str(path), compile=False)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        scheme = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.log_params(self.config.all_params)
        mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

        
        tmpdir = tempfile.mkdtemp()
        export_path = Path(tmpdir) / "saved_model"
        self.model.export(str(export_path))

        
        run_kwargs = {"nested": True} if mlflow.active_run() else {}
        with mlflow.start_run(**run_kwargs) as run:
            mlflow.log_artifacts(str(export_path), artifact_path="model")
            if scheme != "file":
                model_uri = f"runs:/{run.info.run_id}/model"
                mlflow.register_model(model_uri, "VGG16Model")

        shutil.rmtree(tmpdir)


In [24]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 52 images belonging to 2 classes.


d:\anaconda\envs\mlops311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.3122
INFO:tensorflow:Assets written to: C:\Users\Lenovo\AppData\Local\Temp\tmp2n_e1506\saved_model\assets


INFO:tensorflow:Assets written to: C:\Users\Lenovo\AppData\Local\Temp\tmp2n_e1506\saved_model\assets


Saved artifact at 'C:\Users\Lenovo\AppData\Local\Temp\tmp2n_e1506\saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')]
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2503868090896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868090128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868088784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868091472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868089168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868088592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868095120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868089552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868089936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2503868096464: TensorSpec(shape=(), dtype=tf.resource, 

Successfully registered model 'VGG16Model'.
2025/06/24 21:22:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
